## Calculate Bulk Wind Shear for JSON Sounding Files

In [2]:
import metpy.calc as mpcalc
from metpy.units import units

import pandas as pd
import numpy as np

from metpy.calc.tools import get_layer, _get_bound_pressure_height
from metpy.calc.tools import _less_or_close, _greater_or_close, log_interpolate_1d

In [3]:
df = pd.read_json("data_IAD.json")
df

,pres,hght,tmpc,dwpt,wdir,wspd
0,"[[995, 984, 972, 939.7, 925, 911, 908.4, 877.6...","[[46, 93, 194, 305, 610, 752, 888, 914, 1219, ...","[[36, 35, 34, 31.1, 29.8, 28.6, 28.4, 26.1, 24...","[[23, 25, 24.4, 22.6, 21.8, 21.6, 21.4, 19.1, ...","[[210, 210, 210, 210, 215, 223, 225, 270, 280,...","[[7, 9, 12, 12, 10, 8, 8, 9, 12, 15, 18, 21, 2..."


In [4]:
p = df['pres'].values[0][0] * units.hPa
t = df['tmpc'].values[0][0] * units.degC
td = df['dwpt'].values[0][0] * units.degC
dir = df['wdir'].values[0][0] * units.degrees
spd = df['wspd'].values[0][0] * units.knots
heights = df['hght'].values[0][0] * units.meter

In [11]:
u, v = mpcalc.wind_components(df["wspd"].values[0][0] * units.knot, df["wdir"].values[0][0] * units.deg)

## Quick function to get bulk shear at desired layer (meters) for JSON files


In [75]:
def get_bulk_shear(sound_file,depth):
    """Get bulk shear for desired layer depth based on JSON sounding file
    
    Args
    ----
    sound_file : str
        JSON sounding file name
    
    depth : int
        layer depth desired in meters
    
    Returns
    -------
    Prints u, v, speed, and direction for bilk shear values
    
    u_bulk_shear : pint.quantity.build_quantity_class.<locals>.Quantity
        u-component of layer bulk shear
        
    v_bulk_shear : pint.quantity.build_quantity_class.<locals>.Quantity
        v-component of layer bulk shear
        
    bulk_shear_speed : pint.quantity.build_quantity_class.<locals>.Quantity
        layer bulk shear wind speed
        
    bulk_shear_dir : pint.quantity.build_quantity_class.<locals>.Quantity
        layer bulk shear wind direction
        
    """
    printmd(f"\n**Sounding Location: {sound_file}**")
    print(f"Desired layer: {depth/1000}km\n"+\
         "---------------------------------")
    
    df = pd.read_json(sound_file)
    p = df['pres'].values[0][0] * units.hPa
    Z = df['hght'].values[0][0] * units.meter
    
    def replace_empty_str(col):
        for i in range(len(df[col][0][0][:])):
            if df[col][0][0][i] == '':
                df[col][0][0][i] = 0
        return df
    
    for i in df.columns:
        replace_empty_str(i)
        
    u, v = mpcalc.wind_components(df["wspd"].values[0][0] * units.knot, df["wdir"].values[0][0] * units.deg)
    u_bulk_shear,v_bulk_shear = mpcalc.bulk_shear(p,u,v,heights=Z,depth=depth * units.meter)
    print(f"u-bulk shear: {u_bulk_shear}\nv-bulk shear: {v_bulk_shear}")
    
    bulk_shear_speed = np.sqrt(u_bulk_shear**2 + v_bulk_shear**2)
    bulk_shear_dir = mpcalc.wind_direction(u_bulk_shear,v_bulk_shear)
    print(f"bulk shear speed: {bulk_shear_speed}\nbulk shear direction: {bulk_shear_dir}")
    
    return u_bulk_shear, v_bulk_shear, bulk_shear_speed, bulk_shear_dir

In [322]:
def get_layer(pressure, *args, heights=None, bottom=None, depth=100 * units.hPa,
              interpolate=True):
    r"""Return an atmospheric layer from upper air data with the requested bottom and depth.

    This function will subset an upper air dataset to contain only the specified layer. The
    bottom of the layer can be specified with a pressure or height above the surface
    pressure. The bottom defaults to the surface pressure. The depth of the layer can be
    specified in terms of pressure or height above the bottom of the layer. If the top and
    bottom of the layer are not in the data, they are interpolated by default.

    Parameters
    ----------
    pressure : array-like
        Atmospheric pressure profile
    args : array-like
        Atmospheric variable(s) measured at the given pressures
    heights: array-like, optional
        Atmospheric heights corresponding to the given pressures. Defaults to using
        heights calculated from ``p`` assuming a standard atmosphere [NOAA1976]_.
    bottom : `pint.Quantity`, optional
        The bottom of the layer as a pressure or height above the surface pressure. Defaults
        to the highest pressure or lowest height given.
    depth : `pint.Quantity`, optional
        The thickness of the layer as a pressure or height above the bottom of the layer.
        Defaults to 100 hPa.
    interpolate : bool, optional
        Interpolate the top and bottom points if they are not in the given data. Defaults
        to True.

    Returns
    -------
    `pint.Quantity, pint.Quantity`
        The pressure and data variables of the layer

    """
    # If we get the depth kwarg, but it's None, set it to the default as well
    if depth is None:
        depth = 100 * units.hPa

    # Make sure pressure and datavars are the same length
    for datavar in args:
        if len(pressure) != len(datavar):
            raise ValueError('Pressure and data variables must have the same length.')

    # If the bottom is not specified, make it the surface pressure
    if bottom is None:
        bottom = np.nanmax(pressure.m) * pressure.units

    bottom_pressure, bottom_height = _get_bound_pressure_height(pressure, bottom,
                                                                heights=heights,
                                                                interpolate=interpolate)

    # Calculate the top if whatever units depth is in
    if depth.dimensionality == {'[length]': -1.0, '[mass]': 1.0, '[time]': -2.0}:
        top = bottom_pressure - depth
    elif depth.dimensionality == {'[length]': 1}:
        top = bottom_height + depth
    else:
        raise ValueError('Depth must be specified in units of length or pressure')

    top_pressure, _ = _get_bound_pressure_height(pressure, top, heights=heights,
                                                 interpolate=interpolate)

    ret = []  # returned data variables in layer

    # Ensure pressures are sorted in ascending order
    sort_inds = np.argsort(pressure)
    pressure = pressure[sort_inds]

    # Mask based on top and bottom pressure
    inds = (_less_or_close(pressure, bottom_pressure)
            & _greater_or_close(pressure, top_pressure))
    p_interp = pressure[inds]

    # Interpolate pressures at bounds if necessary and sort
    if interpolate:
        # If we don't have the bottom or top requested, append them
        if not np.any(np.isclose(top_pressure, p_interp)):
            p_interp = np.sort(np.append(p_interp.m, top_pressure.m)) * pressure.units
        if not np.any(np.isclose(bottom_pressure, p_interp)):
            p_interp = np.sort(np.append(p_interp.m, bottom_pressure.m)) * pressure.units

    ret.append(p_interp[::-1])

    for datavar in args:
        # Ensure that things are sorted in ascending order
        datavar = datavar[sort_inds]

        if interpolate:
            # Interpolate for the possibly missing bottom/top values
            datavar_interp = log_interpolate_1d(p_interp, pressure, datavar)
            datavar = datavar_interp
        else:
            datavar = datavar[inds]

        ret.append(datavar[::-1])
    return ret

In [7]:
from metpy import constants as mpconsts

t0 = 288. * units.kelvin
p0 = 1013.25 * units.hPa

def height_to_pressure_std(height):
    r"""Convert height data to pressures using the U.S. standard atmosphere [NOAA1976]_.

    The implementation inverts the formula outlined in [Hobbs1977]_ pg.60-61.

    Parameters
    ----------
    height : `pint.Quantity`
        Atmospheric height

    Returns
    -------
    `pint.Quantity`
        The corresponding pressure value(s)

    Notes
    -----
    .. math:: p = p_0 e^{\frac{g}{R \Gamma} \text{ln}(1-\frac{Z \Gamma}{T_0})}

    """
    
    gamma = 6.5 * units('K/km')
    return p0 * (1 - (gamma / t0) * height) ** (mpconsts.g / (mpconsts.Rd * gamma))

def pressure_to_height_std(pressure):
    r"""Convert pressure data to heights using the U.S. standard atmosphere [NOAA1976]_.

    The implementation uses the formula outlined in [Hobbs1977]_ pg.60-61.

    Parameters
    ----------
    pressure : `pint.Quantity`
        Atmospheric pressure

    Returns
    -------
    `pint.Quantity`
        The corresponding height value(s)

    Notes
    -----
    .. math:: Z = \frac{T_0}{\Gamma}[1-\frac{p}{p_0}^\frac{R\Gamma}{g}]

    """
    gamma = 6.5 * units('K/km')
    return (t0 / gamma) * (1 - (pressure / p0).to('dimensionless')**(
        mpconsts.Rd * gamma / mpconsts.g))


In [9]:
_get_bound_pressure_height(p, 6000*units.m, heights=heights, interpolate=True)

(array([487]) <Unit('hectopascal')>, 6000 <Unit('meter')>)

In [76]:
p_layer, t_layer, u_layer, v_layer, dir_layer, spd_layer, hghts_layer = get_layer(
    p, t, u, v, dir, spd, heights, heights=heights, depth=6000*units.meter)

printmd("**Top of 6km layer (from get_layer)**")
print("==================================")
print("Pressure: " + str(p_layer[-1]))
print("Height (from sounding): " + str(hghts_layer[-1]))
print("Height (from std atm): " + str(pressure_to_height_std(p_layer[-1])))
print()

printmd("**Expected top of 6km layer**")
print("==========================")
#baseHght = pressure_to_height_std(p[0])
baseHght = heights[0]
pressure_bound, height_bound = _get_bound_pressure_height(p, 6000*units.meter+baseHght)
stdPres = height_to_pressure_std(6000*units.meter+baseHght)
stdHght = pressure_to_height_std(stdPres)
print("Pressure bounds from _get_bound_pressure_height: " + str(pressure_bound))
print("Height bounds from _get_bound_pressure_height: " + str(height_bound))
print("Base Height (from std atm): " + str(baseHght))
print("6km Pressure (from std atm): " + str(stdPres))
print("6km-Pressure Height (from std atm): " + str(stdHght))
print()

printmd("**Bottom of 6km layer**")
print("=================")
print("Pressure: " + str(p_layer[0]))
print("Heights (from sounding): " + str(hghts_layer[0]))
print("Heights (from std atm): " + str(pressure_to_height_std(p_layer[0])))
print()

ushr = u_layer[-1] - u_layer[0]
vshr = v_layer[-1] - v_layer[0]

print("U-Shear component: " + str(ushr))
print("V-Shear component: " + str(vshr))
print("Bulk Shear Speed: " + str(np.sqrt(ushr**2 + vshr**2)))

u_bulk_shear, v_bulk_shear, bulk_shear_speed, bulk_shear_dir = get_bulk_shear("data_IAD.json", 6000)

**Top of 6km layer (from get_layer)**

Pressure: 486.0 hectopascal
Height (from sounding): 6083.279983975857 meter
Height (from std atm): 5.780369569654552 kilometer



**Expected top of 6km layer**

Pressure bounds from _get_bound_pressure_height: 468.6454879416241 hectopascal
Height bounds from _get_bound_pressure_height: 6046 meter
Base Height (from std atm): 46 meter
6km Pressure (from std atm): 468.6454879416241 hectopascal
6km-Pressure Height (from std atm): 6.046000000000002 kilometer



**Bottom of 6km layer**

Pressure: 995.0 hectopascal
Heights (from sounding): 46.0 meter
Heights (from std atm): 0.15295996804345655 kilometer

U-Shear component: 21.819830420597967 knot
V-Shear component: -27.258009986877884 knot
Bulk Shear Speed: 34.91567138160724 knot



**Sounding Location: data_IAD.json**

Desired layer: 6.0km
---------------------------------
u-bulk shear: 21.819830420597967 knot
v-bulk shear: -27.258009986877884 knot
bulk shear speed: 34.91567138160724 knot
bulk shear direction: 321.32299241704106 degree


In [53]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [452]:
def calc_BRN(pressure, u, v, temp, cape, heights=None):
    r"""Calculate Bulk Richardson Number and BRN shear.

    Parameters
    ----------
    pressure : `pint.Quantity`
                Atmospheric pressure profile
    u        : `pint.Quantity`
                U-component of wind.
    v        : `pint.Quantity`
                V-component of wind.
    temp     : `pint.Quantity`
                Atmospheric temperature profile (can be either temperature or virtual temperature)
    cape     : `pint.Quantity`
                CAPE value to use as the numerator of the BRN calculation.
    heights  : `pint.Quantity`, optional
                Heights in meters from sounding (not AGL)

    Returns
    -------
    brn     : `pint.Quantity`
               Bulk Richadson Number
    brn_shr : `pint.Quantity`
               Bulk Richardson Shear (the denominator of the BRN)

    """
    
    from metpy import constants as mpconsts
    
    baseZ = heights[0]
    sumZ = -baseZ
    rho6km = 0
    u6km = 0
    v6km = 0
    u500 = 0
    v500 = 0

    for i, p in enumerate(pressure):
        rho = (p.to('Pa')/(temp[i].to('K')*mpconsts.Rd)).to('kg / m^3')
        print(rho)
        sumZ += heights[i]
        u_weighted = u[i]*rho
        v_weighted = v[i]*rho
        u_weighted = u_weighted.magnitude
        v_weighted = v_weighted.magnitude
        
        if (sumZ >= 6000 * units.meter):
            layerZ=heights[i] - heights[i-1]
            fraction = (6000 * units.meter - layerZ)/sumZ
            fraction = fraction.to('dimensionless')
            u_frac = ((u_weighted - uprev) * fraction) + uprev
            v_frac = ((v_weighted - vprev) * fraction) + vprev

            rho6km += rho
            u_weighted = u_frac*rho
            v_weighted = v_frac*rho
            u_weighted = u_weighted.magnitude
            v_weighted = v_weighted.magnitude
            u6km += u_weighted
            v6km += v_weighted
            break

        rho6km += rho
        u6km += u_weighted
        v6km += v_weighted

        if (sumZ < 500 * units.meter):
            u500 += u_weighted
            v500 += v_weighted
            divisor = i+1

        uprev = u_weighted
        vprev = v_weighted

    u6kmAvg = u6km/(i+1)
    v6kmAvg = v6km/(i+1)
    print(u6kmAvg, v6kmAvg)
    u500Avg = u500/divisor
    v500Avg = v500/divisor
    print(u500Avg, v500Avg)
    uDiff = (u6kmAvg-u500Avg)
    vDiff = (v6kmAvg-v500Avg)
    print(uDiff, vDiff)
    mag = np.sqrt(uDiff**2+vDiff**2)
    brnshr = (mag**2)*.5
    brnshr = brnshr.to('m^2/s^2')
    brn = cape/brnu
    brn = brn.magnitude

    return brn, brnshr

In [453]:
calc_BRN(p,u,v,t,5065 * units('J/kg'),heights=heights)

1.1212028197954187 kilogram / meter ** 3
1.1124058850537535 kilogram / meter ** 3
1.1024174949009757 kilogram / meter ** 3
1.0759423235482144 kilogram / meter ** 3
1.063655836068516 kilogram / meter ** 3
1.0517231890518561 kilogram / meter ** 3
1.0494171181571672 kilogram / meter ** 3
1.0216280462859253 kilogram / meter ** 3
0.9964913714755524 kilogram / meter ** 3
0.9686903983105212 kilogram / meter ** 3
7.3759664942045475 5.234657939223949
5.181513773810572 8.974645116357859
2.1944527203939757 -3.7399871771339104


AttributeError: 'numpy.float64' object has no attribute 'to'

In [450]:
def calc_BRN_2(pressure, u, v, temp, cape, heights=None):
    r"""Calculate Bulk Richardson Number and BRN shear.

    Parameters
    ----------
    pressure : `pint.Quantity`
                Atmospheric pressure profile
    u        : `pint.Quantity`
                U-component of wind.
    v        : `pint.Quantity`
                V-component of wind.
    temp     : `pint.Quantity`
                Atmospheric temperature profile (can be either temperature or virtual temperature)
    cape     : `pint.Quantity`
                CAPE value to use as the numerator of the BRN calculation.
    heights  : `pint.Quantity`, optional
                Heights in meters from sounding (not AGL)

    Returns
    -------
    brn     : `pint.Quantity`
               Bulk Richadson Number
    brn_shr : `pint.Quantity`
               Bulk Richardson Shear (the denominator of the BRN)

    """
    
    from metpy import constants as mpconsts
    
    baseZ = heights[0]
    sumZ = -baseZ
    rho6km = 0
    u6km = 0
    v6km = 0
    u500 = 0
    v500 = 0

    rho = (pressure[0].to('Pa')/(temp[0].to('K')*mpconsts.Rd)).to('kg / m^3')
    u_weighted = u[0]*rho
    v_weighted = v[0]*rho
    u_weighted = u_weighted.magnitude
    v_weighted = v_weighted.magnitude
    ubase_weighted = u_weighted
    vbase_weighted = v_weighted

    for i, p in enumerate(pressure):
        rho = (p.to('Pa')/(temp[i].to('K')*mpconsts.Rd)).to('kg / m^3')
        print(rho)
        sumZ += heights[i]
        u_weighted = u[i]*rho
        v_weighted = v[i]*rho
        u_weighted = u_weighted.magnitude
        v_weighted = v_weighted.magnitude
        
        if (sumZ >= 6000 * units.meter):
            layerZ=heights[i] - heights[i-1]
            fraction = (6000 * units.meter - layerZ)/sumZ
            fraction = fraction.to('dimensionless')
            u_frac = ((u_weighted - uprev) * fraction) + uprev
            v_frac = ((v_weighted - vprev) * fraction) + vprev

            u_weighted = u_frac*rho
            v_weighted = v_frac*rho
            u_weighted = u_weighted.magnitude
            v_weighted = v_weighted.magnitude
            u6km += u_weighted
            v6km += v_weighted
            
            break

        if (sumZ < 500 * units.meter):
            u500 += u_weighted
            v500 += v_weighted
            divisor = i+1

        uprev = u_weighted
        vprev = v_weighted

    u6kmAvg = u6km/(i+1)
    v6kmAvg = v6km/(i+1)
    print(u6kmAvg, v6kmAvg)
    u500Avg = u500/divisor
    v500Avg = v500/divisor
    print(u500Avg, v500Avg)
    ushr = (u6kmAvg-u500Avg)
    vshr = (v6kmAvg-v500Avg)
    print(ushr, vshr)
    mag = np.sqrt(ushr**2+vshr**2)
    print(mag)
#    brnu = (mag)*.5
    brnshr = (mag**2)*.5
    brnshr = brnshr.to('m^2/s^2')
    brn = cape/brnshr
    brn = brn.magnitude

    return brn, brnshr

In [451]:
calc_BRN_2(p,u,v,t,5065 * units('J/kg'),heights=heights)

1.1212028197954187 kilogram / meter ** 3
1.1124058850537535 kilogram / meter ** 3
1.1024174949009757 kilogram / meter ** 3
1.0759423235482144 kilogram / meter ** 3
1.063655836068516 kilogram / meter ** 3
1.0517231890518561 kilogram / meter ** 3
1.0494171181571672 kilogram / meter ** 3
1.0216280462859253 kilogram / meter ** 3
0.9964913714755524 kilogram / meter ** 3
0.9686903983105212 kilogram / meter ** 3
1.3013121276200768 -0.4485221039445009
5.181513773810572 8.974645116357859
-3.8802016461904953 -9.42316722030236
10.190782368242408


AttributeError: 'numpy.float64' object has no attribute 'to'

In [456]:
def calc_BRN_3(pressure, u, v, cape, heights=None):
    r"""Calculate Bulk Richardson Number and BRN shear.

    Parameters
    ----------
    pressure : `pint.Quantity`
                Atmospheric pressure profile
    u        : `pint.Quantity`
                U-component of wind.
    v        : `pint.Quantity`
                V-component of wind.
    cape     : `pint.Quantity`
                CAPE value to use as the numerator of the BRN calculation.
    heights  : `pint.Quantity`, optional
                Heights in meters from sounding (not AGL)

    Returns
    -------
    brn     : `pint.Quantity`
               Bulk Richadson Number
    brn_shr : `pint.Quantity`
               Bulk Richardson Shear (the denominator of the BRN)

    """

    p_layer500, u_layer500, v_layer500 = get_layer(p, u, v, heights=None, depth=500*units.meter)
    u_layer500 = u_layer500.to('m/s')
    v_layer500 = v_layer500.to('m/s')
    avgU500 = sum(u_layer500 * p_layer500) / sum(p_layer500)
    avgV500 = sum(v_layer500 * p_layer500) / sum(p_layer500)
    print(avgU500, avgV500)
    
    p_layer6km, u_layer6km, v_layer6km = get_layer(p, u, v, heights=None, depth=6000*units.meter)
    u_layer6km = u_layer6km.to('m/s')
    v_layer6km = v_layer6km.to('m/s')
    avgU6km = sum(u_layer6km * p_layer6km) / sum(p_layer6km)
    avgV6km = sum(v_layer6km * p_layer6km) / sum(p_layer6km)
    print(avgU6km, avgV6km)

    ushr = avgU6km-avgU500
    vshr = avgV6km-avgV500
    print(ushr, vshr)
    
    mag = np.sqrt(ushr**2+vshr**2)
    print(mag)
    
    brnshr = (mag**2)*.5
    brn = cape/brnshr
    brn = brn.magnitude

    return brn, brnshr

In [457]:
calc_BRN_3(p,u,v,5065 * units('J/kg'),heights=heights)

2.6598553905049123 meter / second 4.577493242528277 meter / second
8.234877431680705 meter / second -2.8785490523619663 meter / second
5.575022041175792 meter / second -7.456042294890244 meter / second
9.309857005496275 meter / second


(116.87548453756854, 43.33671873139403 <Unit('meter ** 2 / second ** 2')>)

In [446]:
def calc_BRN_4(pressure, spd, cape, heights=None):
    r"""Calculate Bulk Richardson Number and BRN shear.

    Parameters
    ----------
    pressure : `pint.Quantity`
                Atmospheric pressure profile
    spd        : `pint.Quantity`
                Wind speed.
    cape     : `pint.Quantity`
                CAPE value to use as the numerator of the BRN calculation.
    heights  : `pint.Quantity`, optional
                Heights in meters from sounding (not AGL)

    Returns
    -------
    brn     : `pint.Quantity`
               Bulk Richadson Number
    brn_shr : `pint.Quantity`
               Bulk Richardson Shear (the denominator of the BRN)

    """

    p_layer500, spd_layer500 = get_layer(p, spd, heights=None, depth=500*units.meter)
    avg500 = sum(spd_layer500 * p_layer500) / sum(p_layer500)
    print(avg500)
    
    p_layer6km, spd_layer6km = get_layer(p, spd, heights=None, depth=6000*units.meter)
    avg6km = sum(spd_layer6km * p_layer6km) / sum(p_layer6km)
    print(avg6km)

    shr = avg6km-avg500
    print(shr)

    print(spd_layer6km)
    
    brnshr = (shr**2)*.5
    brnshr = brnshr.to('m^2/s^2')
    brn = cape/brnshr
    brn = brn.magnitude

    return brn, brnshr

In [447]:
calc_BRN_4(p, spd, 5065 * units('J/kg'), heights=heights)

10.292262720773174 knot
19.41731217708562 knot
9.125049456312446 knot
[7.0 9.0 12.0 12.0 10.0 8.0 8.0 9.0 12.0 15.0 18.0 21.0 21.0 22.0 22.0 24.0 24.0 23.0 22.0 22.0 22.0 23.0 23.0 30.0 30.0 31.0 31.0 32.0 33.0 33.0 34.0 34.0 34.741301661987926] knot


(459.6867949666405, 11.01837175977084 <Unit('meter ** 2 / second ** 2')>)

In [403]:
def calc_BRN_5(pressure, u, v, cape, heights=None):
    r"""Calculate Bulk Richardson Number and BRN shear.

    Parameters
    ----------
    pressure : `pint.Quantity`
                Atmospheric pressure profile
    u        : `pint.Quantity`
                U-component of wind.
    v        : `pint.Quantity`
                V-component of wind.
    cape     : `pint.Quantity`
                CAPE value to use as the numerator of the BRN calculation.
    heights  : `pint.Quantity`, optional
                Heights in meters from sounding (not AGL)

    Returns
    -------
    brn     : `pint.Quantity`
               Bulk Richadson Number
    brn_shr : `pint.Quantity`
               Bulk Richardson Shear (the denominator of the BRN)

    """

    p_layer500, u_layer500, v_layer500 = get_layer(p, u, v, heights=None, depth=500*units.meter)
    print(u_layer500, v_layer500)
    avgU500 = sum(u_layer500) / p_layer500.size
    avgV500 = sum(v_layer500) / p_layer500.size
    print(avgU500, avgV500)
    
    p_layer6km, u_layer6km, v_layer6km = get_layer(p, u, v, heights=None, depth=6000*units.meter)
    print(u_layer6km, v_layer6km, p_layer6km)
    avgU6km = sum(u_layer6km) / p_layer6km.size
    avgV6km = sum(v_layer6km) / p_layer6km.size
    print(avgU6km, avgV6km)

    ushr = avgU6km-avgU500
    vshr = avgV6km-avgV500
    print(ushr, vshr)
    
    mag = np.sqrt(ushr**2+vshr**2)
    print(mag)
    
    brnshr = (mag**2)*.5
    brn = cape/brnshr
    brn = brn.magnitude

    return brn, brnshr

In [404]:
calc_BRN_5(p,u,v,5534 * units('J/kg'),heights=None)

[3.500000000000001 4.500000000000001 6.000000000000002 6.000000000000002 5.955203263159421] knot [6.06217782649107 7.794228634059947 10.392304845413264 10.392304845413264 10.01919862464582] knot
5.1910406526318855 knot 8.932042955204674 knot
[3.500000000000001 4.500000000000001 6.000000000000002 6.000000000000002 5.735764363510461 5.455986880499987 5.65685424949238 9.0 11.817693036146498 14.095389311788624 17.38666487320323 19.733545036504076 19.733545036504076 19.05255888325765 18.857680615446473 19.659649062935802 20.12809363069018 20.30779463575532 20.801408663184972 21.250368178359505 20.673237657289985 20.845079101842945 20.845079101842945 24.574561328669756 24.574561328669756 25.079526825623375 24.757700811466087 25.216344115415097 25.27946662292628 25.27946662292628 27.153607341607966 28.187277466871414 29.531685895191128] knot [6.06217782649107 7.794228634059947 10.392304845413264 10.392304845413264 8.191520442889917 5.8508296129533655 5.6568542494923815 1.7763568394002505e-15 

(24.759620990876353, 223.50907560496253 <Unit('knot ** 2')>)

In [458]:
def calc_BRN_6(pressure, u, v, temp, cape, heights=None):
    r"""Calculate Bulk Richardson Number and BRN shear.

    Parameters
    ----------
    pressure : `pint.Quantity`
                Atmospheric pressure profile
    u        : `pint.Quantity`
                U-component of wind.
    v        : `pint.Quantity`
                V-component of wind.
    temp     : `pint.Quantity`
                Atmospheric temperature profile
    cape     : `pint.Quantity`
                CAPE value to use as the numerator of the BRN calculation.
    heights  : `pint.Quantity`, optional
                Heights in meters from sounding (not AGL)

    Returns
    -------
    brn     : `pint.Quantity`
               Bulk Richadson Number
    brn_shr : `pint.Quantity`
               Bulk Richardson Shear (the denominator of the BRN)

    """

    p_layer500, u_layer500, v_layer500, t_layer500 = get_layer(p, u, v, t, heights=None, depth=500*units.meter)
    rho_layer500 = (p_layer500.to('Pa')/(t_layer500.to('K')*mpconsts.Rd)).to('kg/m^3')
    print(rho_layer500)
    
    u_layer500 = u_layer500.to('m/s')
    v_layer500 = v_layer500.to('m/s')
    
    avgU500 = sum(u_layer500 * rho_layer500) / sum(rho_layer500)
    avgV500 = sum(v_layer500 * rho_layer500) / sum(rho_layer500)
    print(avgU500, avgV500)
    
    p_layer6km, u_layer6km, v_layer6km, t_layer6km = get_layer(p, u, v, t, heights=None, depth=6000*units.meter)
    rho_layer6km = (p_layer6km.to('Pa')/(t_layer6km.to('K')*mpconsts.Rd)).to('kg/m^3')
    print(rho_layer6km)
    
    u_layer6km = u_layer6km.to('m/s')
    v_layer6km = v_layer6km.to('m/s')
    
    avgU6km = sum(u_layer6km * rho_layer6km) / sum(rho_layer6km)
    avgV6km = sum(v_layer6km * rho_layer6km) / sum(rho_layer6km)
    print(avgU6km, avgV6km)

    ushr = avgU6km-avgU500
    vshr = avgV6km-avgV500
    print(ushr, vshr)
    
    mag = np.sqrt(ushr**2+vshr**2)
    print(mag)
    
    brnshr = (mag**2)*.5
    brnshr = brnshr.to("m^2/s^2")
    brn = cape/brnshr
    brn = brn.magnitude

    return brn, brnshr

In [464]:
calc_BRN_6(p,u,v,t,5534 * units('J/kg'),heights=None)

[1.1212028197954187 1.1124058850537535 1.1024174949009757 1.0759423235482144 1.0738480240513477] kilogram / meter ** 3
2.66281996716048 meter / second 4.58238029054885 meter / second
[1.1212028197954187 1.1124058850537535 1.1024174949009757 1.0759423235482144 1.063655836068516 1.0517231890518561 1.0494171181571672 1.0216280462859253 0.9964913714755524 0.9686903983105212 0.9434123310970598 0.9192458814127077 0.9190049257426126 0.8920289175479302 0.8880401711622702 0.85577526593426 0.8514827581661372 0.8374841380319189 0.8177273526188595 0.8113475273728411 0.7687938166965704 0.7286548692608193 0.7283983909541453 0.6649997777863086 0.6636724329204676 0.6568613700050064 0.6537157077292952 0.6515641008920628 0.6449425145008832 0.6441279296878698 0.6336263559163309 0.623194388962491 0.6180111600294343] kilogram / meter ** 3
8.451420318874625 meter / second -3.1497349061140034 meter / second
5.788600351714145 meter / second -7.732115196662853 meter / second
9.65885601126395 meter / second


(118.63634728770406, 46.64674972316487 <Unit('meter ** 2 / second ** 2')>)

In [469]:
def calc_BRN_7(pressure, u, v, cape, heights=None):
    r"""Calculate Bulk Richardson Number and BRN shear.

    Parameters
    ----------
    pressure : `pint.Quantity`
                Atmospheric pressure profile
    u        : `pint.Quantity`
                U-component of wind.
    v        : `pint.Quantity`
                V-component of wind.
    temp     : `pint.Quantity`
                Atmospheric temperature profile
    cape     : `pint.Quantity`
                CAPE value to use as the numerator of the BRN calculation.
    heights  : `pint.Quantity`, optional
                Heights in meters from sounding (not AGL)

    Returns
    -------
    brn     : `pint.Quantity`
               Bulk Richadson Number
    brn_shr : `pint.Quantity`
               Bulk Richardson Shear (the denominator of the BRN)

    """

    p_layer500, u_layer500, v_layer500 = get_layer(p, u, v, heights=None, depth=500*units.meter)
    u_layer500 = u_layer500.to('m/s')
    v_layer500 = v_layer500.to('m/s')
    
    avgU500 = sum(u_layer500 * p_layer500) / sum(p_layer500)
    avgV500 = sum(v_layer500 * p_layer500) / sum(p_layer500)
    print(avgU500, avgV500)
    
    p_layer6km, u_layer6km, v_layer6km = get_layer(p, u, v, heights=None, depth=6000*units.meter)    
    u_layer6km = u_layer6km.to('m/s')
    v_layer6km = v_layer6km.to('m/s')
    
    avgU6km = sum(u_layer6km * p_layer6km) / sum(p_layer6km)
    avgV6km = sum(v_layer6km * p_layer6km) / sum(p_layer6km)
    print(avgU6km, avgV6km)

    ushr = avgU6km-avgU500
    vshr = avgV6km-avgV500
    print(ushr, vshr)
    
    mag = np.sqrt(ushr**2+vshr**2)
    print(mag)
    
    brnshr = (mag**2)*.5
    brn = cape/brnshr
    brn = brn.magnitude

    return brn, brnshr

In [471]:
calc_BRN_7(p,u,v,5534 * units('J/kg'),heights=None)

2.6598553905049123 meter / second 4.577493242528277 meter / second
8.234877431680705 meter / second -2.8785490523619663 meter / second
5.575022041175792 meter / second -7.456042294890244 meter / second
9.309857005496275 meter / second


(127.69771597846088, 43.33671873139403 <Unit('meter ** 2 / second ** 2')>)